萧何

In [99]:
XH = ["20", "6", "100", "200", "300"]
Cruz = 30
#region
import os
import nbformat
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter.scrolledtext import ScrolledText
import re
import random

class HX:
    
    def __init__(self, root, markdown_content):
        self.root = root
        self.root.title("Markdown Quiz")
        self.root.geometry("800x600")
        self.HEADER_OPTIONS = XH
        # 挖空内容的变量
        self.markdown_content = markdown_content
        self.lines = self.markdown_content.split("\n")
        self.correct_streak = 0
        # 获取所有标题并初始化下拉菜单的选项

        self.terminal_content = self.find_terminal_headers_with_content(self.lines)
        
        # 当前正确答案的内容
        self.correct_answer = ""
        self.current_option = ""

        # 设置显示和控制区域
        self.create_display_area()
        self.create_option_area()

        # 绑定按键
        self.root.bind("<space>", lambda event: self.show_random_options())
        self.root.bind("<Return>", lambda event: self.check_answer())

        # 显示初始的挖空内容和选项
        self.show_new_quiz()

    def find_terminal_headers_with_content(self, lines, selected_headers=None):
        # 提取符合条件的标题内容
        terminal_headers = []
        for i, line in enumerate(lines):
            # 检查当前行是否是标题行
            match = re.match(r"^(#+)\s", line)
            if match:
                content = []

                # 收集当前标题下的内容
                for j in range(i + 1, len(lines)):
                    next_match = re.match(r"^(#+)\s", lines[j])
                    if next_match:  # 如果遇到新的标题，停止收集内容
                        break
                    content.append(lines[j])

                content_str = "\n".join(content).strip()  # 合并内容
                terminal_headers.append((line, content_str))  # 添加标题和内容到结果
        
        if self.HEADER_OPTIONS:
            selected_headers = random.choice(self.HEADER_OPTIONS)
        
        # 如果指定了 selected_headers，则进行筛选
        if selected_headers:
            terminal_headers = [
            (header, content) for header, content in terminal_headers
            if re.match(rf"^#\s*{re.escape(selected_headers)}(?:\s|$)", header)
        ]

            # 处理 header 和 content
            if terminal_headers:
                terminal_headers = [
                    (
                        # 处理 header：删除 '#'，加上 '-'
                        header.lstrip("#").strip() + "\n",
                        # 在 content 开头添加修改后的 header 内容，第一条内容前加两个空格
                        f"- {header.lstrip('#').strip()}\n  {content}"
                    )
                    for header, content in terminal_headers
                ]

        return terminal_headers

    def create_display_area(self):
        # 创建展示Markdown内容的区域，占窗口一半高度
        display_frame = tk.Frame(self.root)
        display_frame.pack(side=tk.TOP, expand=True, fill=tk.BOTH, padx=10, pady=10)
        display_frame.config(height=300)

        self.text_area = ScrolledText(display_frame, wrap=tk.WORD, font=("Helvetica", 16))
        self.text_area.pack(expand=True, fill=tk.BOTH)
        self.text_area.config(state=tk.DISABLED)

    def display_image(self, image_path, max_size=(300, 300)):
        # Enable the text area for modification
        self.text_area.config(state=tk.NORMAL)

        # Load the image
        img = Image.open(image_path)
        original_width, original_height = img.size

        # Calculate scaling factor
        max_width, max_height = max_size
        scale_factor = min(max_width / original_width, max_height / original_height, 1)

        # Compute new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)

        # Resize the image with high-quality resampling
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Convert to PhotoImage for Tkinter
        photo = ImageTk.PhotoImage(img)

        # Create a label to hold the image
        image_label = tk.Label(self.text_area, image=photo, borderwidth=0)
        image_label.photo = photo  # Retain a reference to prevent garbage collection

        # Insert the label as a window in the text widget
        self.text_area.window_create(tk.END, window=image_label)
        self.text_area.insert(tk.END, " ")  # Add a space between images

        # Disable the text area to prevent editing
        self.text_area.config(state=tk.DISABLED)

    def display_a(self, image_path, max_size=(300, 300)):
        # Enable the text area for modification
        self.text_area.config(state=tk.NORMAL)

        # Load the image
        img = Image.open(image_path)
        original_width, original_height = img.size

        # Calculate scaling factor
        max_width, max_height = max_size
        scale_factor = min(max_width / original_width, max_height / original_height, 1)

        # Compute new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)

        # Resize the image with high-quality resampling
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Convert to PhotoImage for Tkinter
        photo = ImageTk.PhotoImage(img)

        # Create a frame to hold the image with a black border
        frame = tk.Frame(self.text_area, bg="black", bd=1)  # Black border with 2-pixel width
        image_label = tk.Label(frame, image=photo, bg="black")  # Label inside the frame
        image_label.photo = photo  # Retain a reference to prevent garbage collection
        image_label.pack(padx=1, pady=1)  # Add padding inside the frame

        # Insert the frame as a window in the text widget
        self.text_area.window_create(tk.END, window=frame)
        self.text_area.insert(tk.END, " ")  # Add a space between images

        # Disable the text area to prevent editing
        self.text_area.config(state=tk.DISABLED)

    def create_option_area(self):
        # 创建选项和按钮区域，占窗口另一半高度
        option_frame = tk.Frame(self.root)
        option_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # 创建选项标签，允许换行显示
        self.option_label = tk.Label(option_frame, text="", font=("Helvetica", 18), width=80, wraplength=600, anchor="center")
        self.option_label.pack(pady=10)

        # 创建按钮容器并固定位置
        button_frame = tk.Frame(option_frame)
        button_frame.pack(pady=10)

        self.check_button = tk.Button(
            button_frame,
            text="Check",
            command=self.check_answer,
            width=10,  # 调整按钮的宽度
            height=2,  # 调整按钮的高度
            font=("Helvetica", 14)  # 调整字体大小
        )
        self.check_button.pack(side=tk.LEFT, padx=20)

        self.next_button = tk.Button(
            button_frame,
            text="Next",
            command=self.show_random_options,
            width=10,  # 调整按钮的宽度
            height=2,  # 调整按钮的高度
            font=("Helvetica", 14)  # 调整字体大小
        )
        self.next_button.pack(side=tk.LEFT, padx=20)

    def show_new_quiz(self):
        # 显示新的挖空内容和正确答案
        self.correct_streak += 1
        if self.terminal_content:
            # 随机选择一个最末端标题及其内容
            random_terminal_header, content = random.choice(self.terminal_content)
            blanked_content, correct_answer = self.find_and_blank_lowest_content(content)

            
            display_content = f"\n\n{blanked_content}"  # 包含标题和挖空后的内容
            self.render_markdown_to_text(display_content)
            self.correct_answer = correct_answer.lstrip("- ").lstrip()
            self.show_random_options()
            # 将标题和内容合并到显示内容中
            correct_answer = correct_answer.lstrip("- ").lstrip()
            print(correct_answer)
            
            entries = [line.lstrip('- ').strip() for line in blanked_content.strip().split('\n') if line.strip()]
            # Add the correct_answer as the last entry

            image_paths = []  # Collect valid image paths here
            image_directory = r"D:\desktop\Pic"
            for entry in entries:
                # Sanitize the entry to create a valid file name
                sanitized_entry = entry.replace(":", "_").replace("/", "_")
                image_path = os.path.join(image_directory, f"{sanitized_entry}.png")
                
                # Check if the image file exists
                if os.path.exists(image_path):
                    image_paths.append(image_path)  # Add valid paths to the list

            # Display all collected images
            for path in image_paths:
                self.display_image(path)  # Assuming self.display_image handles displaying the image
            
            image_a = os.path.join(image_directory, correct_answer + ".png")
            # Display the image if it exists
            if os.path.exists(image_a):
                self.display_a(image_a)

    def show_random_options(self):
        # 获取当前正确答案和三个干扰选项，并随机排列
        if self.correct_answer:
            # 将正确答案和干扰选项组合在一起并打乱顺序
            options = [self.correct_answer] + self.get_random_disturbing_options(num_options=3)
            random.shuffle(options)  # 随机排列选项顺序
            
            # 随机选择其中一个选项显示
            self.current_option = random.choice(options)
            self.option_label.config(text=self.current_option)

    def check_answer(self):
        # 检查当前选项是否正确
        global selected_headers
        if self.current_option == self.correct_answer:
            if self.correct_streak >= Cruz:
                root.destroy()
            else:
                selected_headers = random.choice(self.HEADER_OPTIONS)
                self.terminal_content = self.find_terminal_headers_with_content(self.lines)
                self.show_new_quiz()
        else:
            self.correct_streak = 0

    def render_markdown_to_text(self, content):
        # 在显示区域渲染 Markdown 内容
        self.text_area.config(state=tk.NORMAL)
        self.text_area.delete("1.0", tk.END)
        lines = content.split("\n")

        for line in lines:
            if line.startswith("- "):
                # 列表项
                self.text_area.insert(tk.END, line + "\n")
            else:
                # 普通文本
                self.text_area.insert(tk.END, line + "\n")  # 普通文本无样式

        self.text_area.config(state=tk.DISABLED)

    def find_and_blank_lowest_content(self, content):
        lines = [line for line in content.split("\n") if line.strip()]  # 去掉空行
        non_lowest_items = []
        def get_indent_level(line):
            """计算行的缩进数量（以空格为单位）"""
            indent_level = len(line) - len(line.lstrip())  # 计算缩进级别
            return indent_level

        # 遍历所有行
        for i, line in enumerate(lines):
            if re.match(r"^\s*-\s", line):  # 条目以 - 开头
                current_indent = get_indent_level(line)  # 当前条目的缩进层级
                has_direct_child = False
                
                # 检查后续行，判断是否有直接子项
                for j in range(i + 1, len(lines)):
                    next_line = lines[j]
                    next_indent = get_indent_level(next_line)  # 后续条目的缩进层级
                    
                    if next_indent > current_indent:  # 是子项
                        has_direct_child = True
                        break
                    if next_indent <= current_indent:  # 遇到同级或上级条目，停止检查
                        break

                if has_direct_child:
                    non_lowest_items.append(i)

        if non_lowest_items:
            # 随机选择一个非最低条目
            chosen_index = random.choice(non_lowest_items)
            chosen_line = lines[chosen_index]

            # 找到其直接子项
            child_lines = []
            current_indent = get_indent_level(lines[chosen_index])  # 被选条目的缩进层级
            for j in range(chosen_index + 1, len(lines)):
                next_line = lines[j]
                next_indent = get_indent_level(next_line)

                if next_indent == current_indent + 2:  # 直接子项（假设每一级缩进为2个空格）
                    child_lines.append(next_line)
                if next_indent <= current_indent:  # 遇到同级或上级条目，停止寻找子项
                    break

            # 随机选择一个子项，如果没有子项则设为 None
            chosen_child = random.choice(child_lines) if child_lines else None

            # 向上检索找到所有所属的父级条目，直到 indent_level 为 0
            parent_lines = []
            for k in range(chosen_index - 1, -1, -1):  # 从当前条目向上检索
                prev_line = lines[k]
                prev_indent = get_indent_level(prev_line)
                if prev_indent < current_indent and prev_indent % 2 == 0:  # 找到更高级别的条目
                    parent_lines.append(prev_line)
                    current_indent = prev_indent  # 更新当前缩进级别
                if prev_indent == 0:  # 到顶层条目时停止
                    break

            parent_lines.reverse()  # 保证从顶层条目到当前条目按顺序排列

            # 构造挖空后的内容，包括父级条目、chosen_line 和其子项
            blanked_content = []
            for line in parent_lines:
                blanked_content.append(line)  # 添加所有父级条目
            if chosen_line:
                blanked_content.append(chosen_line)
            for line in child_lines:
                if line == chosen_child:
                    # 只挖空随机选择的子项
                    blanked_content.append(re.sub(r"- .+", "- " + "_" * (len(line) - 2), line))
                else:
                    blanked_content.append(line)
            correct_answer = chosen_child.strip() if chosen_child else None
            
            return "\n".join(blanked_content), correct_answer
        else:
            # 当没有非最低条目时，直接从所有条目中随机挖空
            all_items = [line for line in lines if re.match(r"^\s*-\s", line)]  # 所有符合条目格式的行
            chosen_item = random.choice(all_items) if all_items else None

            # 构造挖空后的内容
            blanked_content = []
            for line in lines:
                if line == chosen_item:
                    # 挖空选中的条目
                    blanked_content.append(re.sub(r"- .+", "- " + "_" * (len(line) - 2), line))
                else:
                    blanked_content.append(line)
        
            # 返回挖空后的内容和答案
            correct_answer = chosen_item.strip() if chosen_item else None
            return "\n".join(blanked_content), correct_answer

    def get_random_disturbing_options(self, num_options=3):
        # 遍历 self.lines，选择最低层级的项目符号行
        all_lowest_options = [
        line.lstrip("- ").lstrip()
        for _, content in self.terminal_content  # 遍历元组，content是第二部分
        for line in content.split("\n")  # 按换行符分割每部分内容
        if re.match(r"^\s*-\s", line)  # 筛选以"- "开头的行
        ]
        # 过滤掉正确答案
        filtered_options = [option for option in all_lowest_options if option != self.correct_answer]
        # 随机选择 num_options 个干扰选项
        disturbing_options = random.sample(filtered_options, num_options)
        return disturbing_options

if __name__ == "__main__":
    # 提取.ipynb文件中的Markdown内容
    filepath = r"D:\desktop\64.ipynb"  # 请调整为您的文件路径
    def extract_markdown_from_ipynb(filepath):
        # 从 .ipynb 文件中提取 Markdown 内容
        with open(filepath, "r", encoding="utf-8") as file:
            notebook = nbformat.read(file, as_version=4)
        markdown_cells = [cell['source'] for cell in notebook.cells if cell['cell_type'] == 'markdown']
        return "\n\n---\n\n".join(markdown_cells)
    markdown_content = extract_markdown_from_ipynb(filepath)

    # 创建并启动Quiz应用
    root = tk.Tk()
    app = HX(root, markdown_content)
    root.mainloop()
#endregion

μ-O,O
~ve
Intensity
below ~200 nm
Transfer to TEM
poor resolution of hyperfine coupling
only radial
R Distance
Trt


# 1 Protein folding
  
  - Why
    - Diseases
      - Replacement, Glu β6 → Val
        - Mutated haemoglobin
        - aggregate into filaments
          - in deoxygenated form
        - causes sickle-cell anaemia
      - Partially unstructured proteins
        - aggregation
      - Neurodegenerative disease
    - Function
    - Design
      - Understanding mechanism
      - Design secondary structure
        - helical bundles
    - Common misfolding
      - α-helix to β-sheet
      - Expose hydrophobic aa
        - leads to aggregation

  - Anfinsen Experiment

    - Ribonuclease A Denaturation
      - 4 disulphide bonds
      - Reagent
        - 8 M
        - urea
          - chaotropic agent
        - β-mercaptoethanol
          - reducing agent
      - Lead to random coil

    - refolded protein
      - √ native activity
      - information necessary for folding
        - aa sequence

    - no activity
      - Remove β-mercaptoethanol only
      - Oxidation Cys
      - Remove urea

    - reform to native
      - trace β-mercaptoethanol

    - native form
      - thermodynamically most stable

  - Golf course landscape
    - Single low energy conformation
    - Random search
      - Very slow folding
    - Levinthal Paradox
      - M^N
      - M
        - #possible conformations
      - N
        - #aa

  - Rugged funnel landscape
    - Reduces conformational search
    - Hierarchical folding
      - structure complexity increases gradually
    - Allows fast folding
      - ms
      - s
    - Small changes in conformation
      - lead to formation
        - few H-bonds
        - hydrophobic contacts
      - decrease in energy
    - Different unfolded molecules
      - via different routes Funnel landscape
    - Trapped misfolded Intermediates
      - productive
        - unfold to refold correctly
      - unproductive
        - Irreversible aggregation
          - through intermolecular interactions

  - Folding equilibrium
    - U(Unfold)
      - Solvation
        - exposed amide
      - Entropy
        - flexible chain

    - N(Native)
      - H-bonded
        - compensate for solvation
      - non-polar residues
        - Hydrophobic effect
      - side chains
        - Van der Waals packing

  - Proteins stabilized by
    - non-bonding interactions
    - e.g.
      - H-bond
      - Hydrophobic
      - Electrostatic
      - Van der Walls

  - Denaturation

    - cooperative process
      - partial unfolding
      - destabilise rest structure

    - Factors
      - Temperature
      - pH
      - Detergents
      - High conc. soluble compound
        - alcohol
      - Salts

    - Effect
      - stability
      - energy barriers

    - Measurment
      - Thermofluor assay
        - Protein Stability
          - Tm
            - melting temperatur
            - midpoint in unfolding process
          - Cofactors, ligands stabilise protein
            - Cu stabilise LPMOs
        - Thermofluor analysis
          - differential scanning fluorimetry

        - Protein + fluorescent dye + cuvette
          - Protein melting
            - fluorescence quenched by water
          - T ↑
          - Protein unfold
            - Dye bind hydrophobic residues
            - Expelling water
            - fluorescence ↑
          - Proiein aggreation
            - Dye dissociafon

# 2

  - Protein composition
    - 50 to 400+ aa
    - L-aa
  
  - PTMs
    - Disulphide bonds
    - Proteolytic cleavage
    - aa Modification
    - Unfolded Modified proteins
      - might not refold
    - Phosphorylation
      - Most abundant PTM
      - in Eukaryotes
    - Glycosylation
      - folding
      - stability
      - signalling

  - Peptide bond

    - resonance
    - rigid
    - planar
    - restricts rotations

  - Conformation preferences

    - Ramachandran plot

      - α-helix
        - φ ≈ -60°
        - ψ ≈ -45°
        - angles close together
        - restricted rotation
          - Cα-Cβ
          - due to R group packing
      - β-sheet
        - φ ≈ -120°
        - ψ ≈ 120°
        - angles farther apart

    - no β-branching
      - prefer α-helix
      - e.g.
        - Ala
        - Glu
        - Lys
        - Met
        - Leu

    - β-branched
      - prefer “extended” polypeptide chain
        - β-sheet
        - steric interactions minimised
      - e.g.
        - Val
        - Ile
        - Thr

    - Conformational search
      - influenced by intrinsic properties
      - adopt favoured
        - φ
        - ψ

    - Chou-Fasman Method

      - empirical method

      - Pref = 1
        - found equally in α, β

      - Pref > 1
        - preference for one

      - Value < ~0.7
        - breaker for region

      - nucleate
        - helix
          - 4/6 residues

        - sheet
          - 3/5 residues

        - region broken
          - average ＜ 1
          - for tetrapeptide

        - P & G
          - irregular
          - flexible loops
          - STOP signal

  - Protecting Group
    
    - Characteristics
      - Easy, selective reaction
      - Not affect aa stereochemistry
      - Stable under coupling conditions
      - Easy to selectively remove without 
        - damaging peptide
        - orthogonal protection

    - Peptide bond formation

      - carboxylic acid + amine → salt

      - requires -COOH activation

      - acylating reagent
        - prepared in separate steps
        - formed in situ
        - e.g.
          - DCC
            - Dicyclohexylcarbodiimide
          - derivatives
            - DIC

    - Side chains protection

      - Essential

        - Lys

        - Ser

        - Thr

        - Tyr

        - Asp

        - Glu

        - Cys

      - Non-essential
        
        - not necessarily

        - yields higher

        - purification steps easier

        - Asn

        - Gln

        - Arg

        - His

        - Trp

# 3 Secondary Structure

  - β-sheet
    - Sequence rich
      - Val
      - Thr
      - Ile
      - Tyr
    - parallel
      - non-contiguous
    - anti-parallel
      - contiguous
    - β-periodicity
      - i to
        - i+2
      - stabilising side chains contacts
    - Intra- and inter-strand interactions
    - P & G
      - STOP signal
      - reverse turn
        - anti-parallel β-hairpin

  - α-helix
    - Sequence rich
      - Ala
      - Leu
      - Lys
      - Glu
    - α-periodicity
      - i to
        - i+3
        - i+4
      - favourable helix stabilising interactions
    - Right-handed helix
    - Rise per turn
      - 5.4 Å pitch
    - R groups point outward
      - influence helical propensity
    - Complementary side chain interactions
      - 3, 7, 11
        - Lys, Glu, Lys
        - Leu, Ile, Leu
    - Amphipathic helix
      - one polar
        - solvent exposed
      - one non-polar face
        - packing against core
    - P & G
      - STOP signal

  - Higher order assembly

    - heptad pseudo
    
    - coiled coils 
      - via intermolecular interactions
      - multiple chains

    - Parallel dimer
      - abcdefg : abcdefg
        - a, d
          - Leu
          - Val
        - b, c, f
          - anything
        - e and g
          - complementary charge

    - Anti-parallel dimer
      - abcdefg : a’b’c’d’e’f’g’
        - a, d
          - Leu
          - Val
        - b, c, f
          - anything
        - e = g
        - e’ = g’

  - De novo design

    - Cys-His-Glu triad
      - in natural ester hydrolases
      - mutated into centre of four-heptad repeat
      - p-Nitrophenyl acetate
        - to p-Nitrophenol
    - Synthetic coiled coils mimic
      - carbonic anhydrase
      - nitrite reductase

    - Non-natural biopolymers

      - Non-natural synthetic
        - oligomers
        - foldamers
        - Different folding rules
      - Peptidomimetics
        - Resistant to proteases

      - 2.5 residues per turn
        - more H-bond
          - C=O (i)
          - NH (i+3)
          - N’H (i+4)
      - Helical arrangement
        - not affected by side chains
      - Oligourea
      
        - H1
        
          - Self-assembly
            - hexameric helix bundle
            - controlled by
              - side chains
            - driven by
              - hydrophobic core formation
          
          - Complementary surface charges
        
        - H2
          - Self-assembly
            - hexameric channel-pore
            - controlled by
              - side chains
            - driven by
              - hydrophilic core formation

          - Surface hydrophobic residues

# 4 Spectroscopic techniques
  
  - Secondary structure
    
    - CD
      - Circular Dichroism
      - Linearly polarised light
        - oscillation
        - in single plane
        - along direction of propagation
      - Circularly polarised light
        - electric vector rotates 
        - left or right
        - around direction of propagation

    - Ellipticity (θ)
      - Chiral molecules 
        - interact differently
          - left light
          - right light
        - producing net absorption
      - proteins
        - peptide backbone
        - “handedness”

    - no net absorption
      - Achiral
      - racemic molecules

    - Absorption bands
      - sensitive to conformation
      - arise from electronic excitation
        - amide chromophores 酰胺色团

    - Conformational analysis

      - Ag+
        - broad spectrum anti-microbial properties
          - Low toxicity for humans
          - AgNO3 used to treat burns
          - Widespread use in medical devices

      - sil operon
        - genes coding, proteins family
        - Bacterial silver resistance developed
          - Active export through an ATPase
          - Effluxer complex
          - Periplasmic chaperon (SilE)

      - SilE
        - induced by Ag+
        - 143 aa
        - Mass spectrometry
          - 4-8 Ag+ bind

      - His-Met separation
        - helical turns
          - i
          - i+3
          - i+7
          - i+11

        - “in phase”
          - on one surface of helix
          - His
          - Met
    
      - Hypothesis
        - Ag+ induce helix formation

      - apo vs. holo

    - probe
      - monitoring folding transitions

  - Tertiary structures

    - Fluorescence spectroscopy

      - monitor protein denaturation
      - intrinsic fluorescence
        - Phe
        - Tyr
        - Trp
      
      - Trp

        - Buried in hydrophobic core

          - blue shift ~20 nm 
            - from 355 nm
            - to 309-335 nm

        - Effect on quantum yield
          - difficult to predict
          - quencher
            - Several aa

    - Linear extrapolation method
      
      - Assumption
        - ΔGeq[D] = ΔGeq + m[D]
          - ΔGeq
            - stability on water alone

      - Transition midpoint
        - U = N
        - Keq = 1
        - ΔGeq[D] = 0
        - midpoint defined
          - [D]_50%
          - ΔGeq = -m[D]_50%

      - Extrapolated ΔG_eq depends on
        - slope m
        - transition midpoint [D]_50%

      - m value
        - hydrophobic surface area
        - larger proteins
          - more co-operative folding transitions
          - bigger m value

        - similar:
          - non-disruptive point mutations

    - Two-state protein folding kinetics
    
      - Equilibrium
        - differences in energy
          - folded
          - unfolded
          - ΔGo_eq

      - Kinetics
        - barrier between states
        - probe protein folds pathway
          - U and N
            - separated by single TS

      - TS
        - Highest point on E landscape
        - All molecules pass
        - × significantly populated

        - Represent balance
          - stabilising interactions
          - entropic cost of restricting flexibility

        - Highly compact？？
          - almost “native-like”
          - highly unfolded

      - Denaturants
        - e.g.
          - Gdm.Cl
            - guanidinium chloride
          - urea
        - protein structure
          - work by stabilising unfolded state
          - unfolded state
            - More amide groups
            - solvent exposed amide groups
          - transition state
            - partially stabilised

          - native folded state
            - least stabilised

          - transition mid-point
            - folded = unfolded

        - Folding kinetics

          - At some [D]
            - free energy UD ＜ ND
            - protein unfolds

          - [D] ↑
            - D bind
              - UD becomes stable
              - UD ↓
            - Barrier to unfolding ΔGDU ↓
            - Barrier to folding ΔGDF ↑

        - Stopped-flow rapid mixing apparatus

          - Collected Kinetic data

          - Rapid jump in conditions
            - triggers refolding

          - Mixing must be rapid 
            - ~2-3 ms
            - compared with folding rate

          - After initial mixing
            - equilibrium re-established
            - molecules refold

          - Detect folding “in real time”
            - using change in fluorescence
            - Repeat exp. at different final [D]

# 6

  - Crystal field theory
    - Axially compressed
      - dz²
      - dx²-y²
      - dxz dyz
      - dxy
    - Axially elongated
      - dx²-y²
      - dz²
      - dxy
      - dxz dyz
    - Square planar
      - dx²-y²
      - dxy
      - dz²
      - dxz dyz

  - DC
    - Random coil
      - ~ve
        - at ~198 nm
      - +ve
        - above ~210 nm
    - β-sheet
      - ~ve
        - at 216-218 nm
      - +ve
        - below ~210 nm
    - α-helix
      - ~ve
        - Strong
        - double band
          - ~210 nm
          - ~222 nm
      - +ve
        - Strong
        - below ~200 nm
    - Proteins
      - linear combination
  
  - coiled coil
    
    - heptad pseudo
    - eg
    - amphipathic
      - amphi
      - pathic

  - Solid Synthesis
    - Coupling to resin
      - Fmoc
    - Deprotection
      - 20% piperidine in DMF
    - Coupling
      - DCC-HOBt
      - DCI-Oxima
    - Capping
      - (CH₃CO)₂O
    - Repeat steps
    - Cleave from resin
      - TFA
  
  - Amino protection

    - Carbamates (urethanes)
      - amide
        - decreases N reactivity
      - ester
        - easily hydrolysed
        - spontaneously decarboxylate
      
      - Boc
        - amine protection
        - deprotection in H+
          - TFA
          - HCl

      - Fmoc
        - stable to H+
          - orthogonal to Boc
        - deprotection in base
          - 20% piperidine in DMF

    - Triphenylmethyl (trityl, Trt) group

      - Trt
        - steric bulk
        - stable to base
          - orthogonal to Fmoc
        - unstable to H+
          - not orthogonal to Boc
        - deprotection in mild H+
          - weak acid
          - acetic acid
          - carbocation stability
    
  - Carboxy protection

    - Esterification

      - methyl, ethyl
        - occurs swiftly
        - stable
        - difficult to deprotect

      - Benzyl
        - resistant to TFA
        - deprotection
          - hydrazinolysis
            - H₂N-NH₂
          - HBr/AcOH
          - hydrogenolysis
            - H₂

      - tBu
        - similar to Boc
        - less sensitive to TFA

      - Phenyl
        - stable to H+
        - deprotection
          - saponification
          - hydrogen peroxide
  
  - Essential
    - Lys
      - Boc
    - Ser
      - tBu
    - Thr
      - tBu
    - Tyr
      - tBu
    - Asp
      - OtBu
    - Glu
      - OtBu
    - Cys
      - Trt
  
  - Non-essential
    - Asn
      - Trt

    - Gln
      - Trt

    - Arg
      - Pbf
      - deprotection with TFA

    - His
      - Trt

    - Trp
      - Boc

# 20

  - Mutations
    
    - Type
      - Point Mutations
      - Silent
        - Same aa
      - Nonsense
        - Shortened sequence
      - Missense
        - Conservative
          - chemically similar
        - Non-Conservative
          - chemically different

    - Sequencing
      - Reagents
        - Template DNA
        - ddNTPs
          - ddATP
          - ddTTP
          - ddGTP
          - ddCTP
        - DNA polymerase
        - Primers

      - Progress
        - Primer Annealing
          - Chain Extension
        - ddNTP Binding
          - Chain Termination
        - Fluorescently Labeled DNA Sample
        - Capillary Gel Electrophoresis
        - Fluorescence Detection
        - Sequence Analysis
          - Reconstruction

  - X-ray crystallography

    - Protein size
      - Broad range

    - Protein type
      - × membrane proteins

    - √
      - total structure determination
      - well established, productive process
    
    - ×
      - “frozen”
      - can appear distorted
      - crystals from soluble proteins

  - NMR spectroscopy
    
    - Five steps
      
      - Bacterial expression system
        - >10 mg
        - labelled
          - 13C
          - 15N

      - Soluble protein
        - quality NMR spectra

      - 2D/3D spectra assignments
        - identify ‘NOE contacts’

      - 1000s of NMR ‘restraints’

      - Structure calculations
        - simulated annealing 
          - restrained molecular dynamics


    - Protein size
      - < 50 kDa

    - Protein type
      - Broad range

    - √
      - studied in aq.
      - protein dynamics possible
    
    - ×
      - often requires stable isotope labelling
      - membrane proteins challenging
      - requires very stable proteins

  - Cryo-EM

    - Step
      - Express protein

      - Purify protein

      - Plunge-freeze
      
        - Dubochet's Vitrification Method
          - sample transferred to metal mesh
          - sample forms thin film 
            - across holes in mesh
            - shot into ethane at -190°
          - water vitrifies around sample
            - cooled by liquid nitrogen
            - in electron microscope

      - Transfer to TEM

      - Image specimen

      - Analyse & reconstruct

        - Frank's Image Analysis for 3D Structures
          - Randomly oriented proteins
            - hit by electron beam
            - leaving trace on image
          - computer discriminates
            - trace
            - fuzzy background
          - Grouping similar traces
            - generates high-resolution 2D image
          - Computer calculation
            - different 2D images relate to each other
            - generates 3D high-resolution structure

    - Protein size
      - > 50 kDa
      - bigger, easier

    - Protein type
      - membrane proteins

    - √
      - near native conditions
      - no requirement for crystallization
      - rapid
      - in situ imaging
    
    - ×
      - destructive e beam
      - complex vitrification
      - lack infrastructure
      - large proteins
        - many replicates
        - receptor-targeting membrane proteins

  - Bioinformatics

    - Define
      - interdisciplinary field 
      - develops methods / software tools
      - understanding biological data
      - large and complex data sets
  
    - Compare alignment scoring matrices
      - BLOSUM62
        - compare sequences similarity
          - likelihood of natural mutations between different residues

      - positive
        - similar residues
        - without affecting structure

      - negative 
        - significant change in residue
        - functional groups
        - mutation from rare aa

    - omics
      - genomics
      - transcriptomics
      - proteomics
      - metabolomics
      - link 
        - transcription
        - translation
        - enzymatic activity
    
    - cutting-edge applications
      - COVID genome sequencing
        - sequence comparison tools
          - viral sequences
            - spread
            - evolution
            - track individual aa mutations
      
      - Predicting protein folding
        - Advantages
          - Rapid screening
          - √ familiar sequences
        - Limitations
          - Proteins with multiple forms
          - × dynamic processes & folding
          - × unusual sequences

  - Protein design

    - Rational design
      - Small scale mutations
        - 10-100
      - based on extensive knowledge
      - Rotamer Confirmations

    - Directed evolution

      - High throughput
        - 10^10-10^12

      - Randomised mutations

      - Step
        - inserted in bacteria as templates
        - produce randomly mutated enzymes
        - test changed enzymes
          - Phage display
            - gene introduced
              - phage DNA inserted into bacteria 
                - produced phage

            - peptide 
              - produced from introduced gene
              - ended up as part of capsule
              - on phage surface

            - get phage
              - antibody designed
                - attach to peptide
              - got peptide gene
        - selected most efficient
        - New random mutations introduced for next

    - De novo design

      - Computational algorithms
        - Exploring Energy Landscape
          - Gradient-based minimization
          - Molecular Dynamics
        - Monte Carlo Methods
          - Metropolis Monte Carlo
          - Monte Carlo Moves
            - Fragment Replacement
            - Rotamer Substitution

      - Based on
        - natural proteins 3D structures
        - folding rules

      - Step

        - Construct multiple-sequence alignment

        - Predict local structure

          - Fragments
            - 20–200 per each segment
        
        - Predict residue contacts
          - Correlated mutations
          - Residue-residue distance constraints
        
        - Assemble 3D models
          - Gradient descent
          - Distance geometry
          - Fragment assembly
        
        - Refine and rank models
          - Landscape projection
          - Refinement trajectory

      - Protein design

        - Fragment assembly
        - Emphasize tightly packed hydrophobic core

      - Protein–protein interactions

        - Protein-protein docking
        - Emphasize hydrogen bonding
          - ensure good hydrophobic packing

      - Ligand-binding proteins

        - Ligand docking
        - Emphasize hydrogen bonding
          - ensure good hydrophobic packing

      - Sensor applications
        - pH switch
        - lon switch
        - Chemically-induced dimerization system
        - Lockr

# 100 X-ray

  - interact with electron density

  - X-ray diffractometer
    - X-ray sensitive digital camera
      - collect the diffraction image
    - mirrors
      - bring the X-rays
      - onto single crystal
    - 3-circle mount
      - maximise positions
  
  - Bio Inorg Chem

    - Binding modes

      - aa
        - Cys
          - S-M
          - M-S-M
        - His
          - δ
          - ε
        - Asp
          - O
          - κ²-O,O
          - μ-O,O

      - Prosthetic Groups

        - Porphyrin
          - Porphyrinate

        - Molybdopterin


    - Crystal field theory
      - Square planar
        - dx²-y²
        - dxy
        - dz²
        - dxz dyz
      - Axially compressed
        - dz²
        - dx²-y²
        - dxz dyz
        - dxy
      - Axially elongated
        - dx²-y²
        - dz²
        - dxy
        - dxz dyz

  - λ = 2dsinθ

  - How Reliable

    - Resolution
    - R factor
      - < 0.25
    - temperature factor, B

  - Structural features

    - 5-6 Å
      - general shape
      - α-helix might identified

    - 3.5-2.5 Å
      - polypeptide back-bone traced

    - 1.5 Å
      - atomic resolution

  - Cu(I) & Cu(II)
    - similar electron densities
    - × distinguish
    - if Higher resolution
      - may see bond distance difference

  - Refining

    - experimental intensity Fo, φc
      - calculated ρ(x,y,z)
        - identify more atoms
        - recalculate
          - φc
          - Fc
    - iterative
    
    - Knowing heavy atom position
      - phase φc
      - intensity Fc
      - reliable structure
        - R factor
          - < 0.25

  - Phase problem approaches

    - Isomorphous replacement
      - Insert heavy metal atom
        - without destroying crystal
        - soaking in heavy metal salt solution

      - Imagine in real space
        - creating 'difference crystal'
        - by subtracting
          - native crystal 
          - from derivative crystal
        - dominated by heavy atom
          - many e

      - Patterson Synthesis
        - resembles an electron density map
        - contains sufficient information
          - locate position of heavy atoms in structure
        - calculation
          - all phases to zero
          - get intensity

# draft
  
  - why X-ray

    - rational drug design
      - structure based functional studies
        - aid effective therapeutic agents and drugs development
      - answers to structure questions
        - global folds
        - atomic details of bonding

    - NMR 
      - no size limitation

    - price
      - a good crystal must be found
      - only limited information 
      about the molecule's dynamic behavior is 
      available from one single diffraction experiment

  - overview
    - Single Crystal Formation
      - single crystal exposed to X-ray source
    - Data Collection
      - detector
        - scattered X-rays captured
        - e.g., film
      - producing diffraction pattern (precession photograph)
    - Data Processing
      - diffraction pattern → electron density map
    - Model Building
      - map + sequence = atomic structure of the protein

  - features of X-ray diffractometer.
  X-ray Source
  rotating copper (Cu) anode 
  generates X-rays
  50 keV electrons
  Mirrors
  X-ray mirrors focus and direct the X-ray beam toward the crystal.
  Crystal Holder
  The crystal is mounted on a rotating platform to expose it to X-rays at different angles.
  Detector
  Detects the diffracted X-rays
  creating a pattern (red dots) that can be analyzed to determine the crystal structure.

  - Eye and Microscope Analogy

    Objects of macroscopic size are visible to us because they scatter
    light falling on them.

    The function of the eye lens is to bring together the light rays to
    form an image on the retina.

    Each scattered light ray has a particular intensity and a particular
    phase. The relative intensities and phases determine the nature of
    the image formed in the eye.

    Information on the shape (structure) of the object is carried in the
    intensities and phases of the light waves scattered by it.

    For smaller objects the eye needs help from more powerful
    lenses. The scattered light is collected by the microscope and
    refracted to bring it into focus in the observer’s eye. It is not
    necessary to collect all of the light but note that the image
    becomes less clear as the proportion of light collected is reduced.

  - Scattering

    The lower limit on the size of objects which can be clearly seen with a
    microscope is determined by the wavelength of light (400 -700 nm).
    Objects that are much smaller than this do not give any significant
    scattering of light.

    To ‘see’ atoms, molecules and proteins (ca. 100 to 1000 times smaller
    than the wavelength of visible light) requires the use of X-rays. In
    principle we require an X-ray microscope in order to observe molecular
    structure. Such an instrument does not exist. WHY?

    The situation is not hopeless because the pattern of scattered X-rays
    can be recorded and then the recombination which is impossible
    physically can be achieved mathematically.

    However, to get an interpretable scattering pattern we must use
    crystalline material WHY?

    When the method is successful it provides and image of the molecular structure. It locates the
    components of the material which interact with the incident X-rays i.e. the electron density.

    The total time taken to record the scattering pattern is usually hours or days.
    WHAT ARE THE IMPLICATIONS OF THIS?

    Some information is lost in the experiment. When the X-ray scattering is recorded the individual
    wave amplitudes are retained as relative intensities but the relative phases are lost ÷ THE
    PHASE PROBLEM.

  - Compare and contrast magnification and diffraction as ways of viewing small objects.

  - Bragg Equation

    - Individual rays have
      - path differences of a whole numberwave lengths (λ) for the waves to 
        - remain in phase

      - leads to
        - constructive interference
        - produce diffraction spot
    
    - Bragg equation show
      - lattice spacing (d) 
      - angle of incidence (θ) 

      - greater θ 
      - smaller d 
      - greater resolution

  - Intensities of Diffracted X-rays

    The intensities of the spots 
    in the diffraction pattern 
    the arrangement of atoms in the crystal structure are 
    related to FOURIER TRANSFORM

      electron density is the Fourier transform of the diffraction pattern.

  - phase problem

    - Some information is lost
    - X-ray scattering is recorded
    - wave amplitudes are retained
    - relative phases are lost

    - approaches

      - Isomorphous replacement
        - insert a heavy metal atom into the protein without destroying the crystal
        - by soaking a protein crystal in a solution containing a heavy metal salt

      - Imagine in real space
        - creating a 'difference crystal' 
        - by subtracting the native crystal from the derivative crystal
        - dominated by the high number of electrons associated with the heavy atom

      - Patterson Synthesis
        - leads to a map that resembles an electron density map. 
        - contains sufficient information to locate the position of the heavy atoms in the structure.
        - calculation, all of the phases to zero
        - use the intensities of this difference diffraction pattern

  - How Reliable

    - resolution
    - R factors
    - temperature factors, B

# 200 XAS

  - X-ray Absorption Spectroscopy

  - pre-edge
    - metal‑ligand interaction
    - covalency of d-orbitals

  - edge
    - metal identity
    - Oxidation State
      - Z_eff

  - EXAFS
    - geometry
    - Coordination Sphere
    - FT
      - R Distance
        - Frequency
        - ± 0.03 Å
      - Intensity
        - Amplitude and phase
        - Coordination number
          - only radial
          - not spatial
        - donor atom identity
          - Heavier atoms, larger amplitudes
  
    - inner coordination shell
      - R Distance
        - ± 0.03 Å
      - NL
        - ± 25%
      - Z
        - ± 1
    
    - bond valence sum analysis
      - oxidation state, V
      - metal-ligand (j) distances, r
      - coordination number

  - Synchrotron Sources
    - insensitive
    - tunable X-rays
    - high intensity X-rays

# draft
  - Amplitude and phase
    - Atom type
    - Only Radial
      - damped sine wave
      - Not Spatial
  - Measurement
    - wavelengths around the absorption “edge” 
      - element of interest

  - Utility
    - electronic information
    - geometric structure information
    - local region around specific elements 
    - in amorphous samples

  - Phenomena Type
    - Absorption edge
      - energy dependent on
        - charge density of absorbing atom 
        - oxidation state

    - Absorption pre-edge
      - energy and intensity
      - covalency of d-orbitals

    - Extended structure

      - EXAFS

  - Extended X-ray Absorption Fine Structure
  
    - Oscillations
      - caused by interference of out-going electron wave 
        - by backscattered electrons from neighbouring atoms
      - pattern of oscillations
        - individual sine waves associated with scattering 
        - caused by each shell of coordinated atoms
    
    - interference (constructive, destructive) depends on
      - X-rays energy (wavelength)
      - nature of scattering atom
      - distance from metal centre

    - Frequency-Distance Relationship
      - longer metal-ligand distance
      
        - more periods of the photelectron wave are required to cover the distance from the metal to the ligand and back

        - takes less change in photoelectron wavelength to go through one period of interference. 
        
        - abosorption is modulated at a higher frequency

    - Amplitude-Coordination Number Relationship

      - coordination with two identical atoms
        - e.g. 
          - two O donors
          - same distance
        - each contribute equally to amplitude of sine wave

      - radial dependence
        - shells of atoms
      - One shell of atoms
        - one damped sine wave

    - Phase and Amplitude-Atom Relationship
      - interaction   
        - outgoing photoelectron wave
        - electron density of a scattering atom, 
      - undergoes a phase shift

      - Variations in electron density (atoms)
        - different phase shifts
      - Heavier atoms 
        - better scatterers of X-rays
        - yeild EXAFS with larger amplitudes

    - Advantages
      - No requirement for crystalline material and 
        - different forms of a system can be investigated
      - Element specific.
      - Samples ~1mM in element can be probed
      - EXAFS provides accurate ± 0.03 Å interatomic distances
      - Studies f(time)

    - Disadvantages
      - Radiation Damage
      - Data analysis complicated
        - extensive fitting required
      - Detailed angular information is not available
      - only gives an average coordination sphere
      - Interatomic distances restricted to the region
        - within ca. 3.5 Å of primary absorber
      - difficult to distinguish
        - N scatterers
        - O scatterers 

    - Interpretation
      - Identifies relationship between
        - oxidation state of the metal, V
        - metal-ligand (j) distances, r
        - coordination number

    - Synchrotron Sources

      - EXAFS technique is inherently insensitive
      - ca. 1% of intensity of edge
      - X-ray source
        - powerful
          - small signal can be extracted from a large background

      - Synchrotron radiation
        - first became available in the 1970s
        - revolutionised 
          - XAS
          - protein crystallography
        - bright
          - high intensity
          - small spot size
          - tunable over 2-30 keV range

    - Bond Valence Sum
      - analyse EXAFS data and
      - predict EXAFS spectra for metal centres in protein


# 300 EPR
  
  - g
    - 2.0023
      - free electron
      - ‘spin only’ value
    - hν = B g μ_B
    - chemical shift

  - A
    - coupling constant
    - Azz
      - larger hyperfine coupling parameter
      - e localized on metal
      - lower covalency

  - EPR spectrometer
    
    - High frequency microwave 
      - good resolution of g values
      - poor resolution of hyperfine coupling

    - Low frequency microwave
      - poor resolution of g values
      - good resolution of hyperfine coupling
    
    - X-band frequency microwave
      - reasonable balance
  
  - S
    - 1/2
      - EPR active
    - 0
      - EPR silent

  - Frozen

    - Axial Spectra
      - gxx = gyy(g⊥) ≠ gzz(g∥)
        - square pyramidal
        - trigonal pyramidal
        - flattened tetrahedron
        - square plane
      - Elongated octahedron
        - gxx = gyy < gzz
        - Axially Elongated
          - g⊥ < g‖
          - unpaired e in dx²-y²
      - Compressed octahedron
        - gxx = gyy > gzz
        - Axially compressed
          - g⊥ > g‖
          - unpaired e in dz²

    - Rhombic Spectra
      - gxx ≠ gyy ≠ gzz
      - Rhombic Low symmetry

  - Fluid

    - Anisotropic molecule
      - tumbling more rapidly
        - than experiment timescale
      - averaged anisotropy

    - Isotropic Spectra
      - gxx = gyy = gzz


  - 2nI+1

# draft

  - Fundamentals

    - Electron Paramagnetic Resonance
      - keep
        - electromagnetic radiation frequency constant (ν)
      - scan
        - magnetic field (Bo)
        - detect dA/dBo
      - h
        - Planck’s constant
      - μB
        - Bohr magneton constant
    - Zeeman Effect

    - unique fingerprints
      - compound identification

  - EPR Spectrometer

    - Microwave Bridge
    - Waveguide
    - Sample Space
    - Magnet
    - Console
    - Microwave Frequency Bands

  - S = ½ Transition Metal

    - unpaired electron
      - associated with at least on transition metal ion

    - anisotropic system
      
      - g no longer equal 2.0023

  - g and orientation Bo

    - EPR signal
      - orientation relative to the magnetic field Bo

    - frozen solution
      - 77 K
    
    - frozen solution spectrum
      - summation all different orientations

    - g = 2.0023
      - g value for a free e
      - ‘spin only’ value

  - e.g.

    - Hyperfine Couplings

      - CuN2O2 coordination sphere

      - I=3/2
        - 63Cu (0.69%)
        - 65Cu (0.31%)
        - 2nI+1 lines
    
      - I=1
        - 14N
        - 2nI+1 lines

      - Cu(II) Protein
        - Type I Blue Copper

# P

  - Free Electrons

# Hallmarks of Cancer

  - Self-Sufficiency in Growth Signals

    - mitogenic growth signals (GS)
      - quiescent state → active proliferative state
      - transmitted
        - transmembrane receptors
      - signaling molecules
        - diffusible growth factors
        - extracellular matrix components
        - cell-to-cell adhesion/interaction molecules
    
      - oncogenes
        - mimicking GS

  Dependence on growth signaling is apparent when propagating normal cells in culture, which typically proliferate only when supplied with appropriate diffusible mitogenic factors and a proper substratum for their integrins. 
  Such behavior contrasts strongly with that of tumor cells, which invariably show a greatly reduced dependence on exogenous growth stimulation. 
  The conclusion is that tumor cells generate many of their own growth signals, thereby reducing their dependence on stimulation from their normal tissue microenvironment. 
  This liberation from dependence on exogenously derived signals disrupts a critically important homeostatic mechanism that normally operates to ensure a proper behavior of the various cell types within a tissue.

# 0 interview

  - VZV

## 50 HCC & CRLM

## 50.0 Acquisition / Processing

  - TCGA database 
    - RNA expression profiles
    - clinical data
    - via UCSC Xena platform

  - survival analysis
    - 368 samples
  
  - Processing
    - TPM format
    - log2-transformed

  - sc

    - sc dataset
      - GEO
      - 10 HCC
      - 9 CRLM

    - Quality control criteria
      - mitochondrial content
        - ＜ 20%
      - UMI counts
        - 200-20,000
      - gene counts
        - 200-5,000

    - Seurat functions 
      - NormalizeData
        - Data normalization
      - FindVariableFeatures
        - selection of highly variable genes
        - 2000 genes
      - ScaleData
        - data transformation
        - eliminating cell cycle effects with the parameters 
        - vars.to.regress = c("S.Score", "G2M.Score")

      - Dimensionality reduction methods 
        - UMAP
        - TSNE
        - Louvain

    - Batch effect correction
      - Harmony

    - Differential gene expression
      - Between
        - clusters
        - cell types
      - FindAllMarkers function
        - p-value < 0.05, log2FC > 0.25
        - expression proportion > 0.1

  - ST

    - HCC ST
      - 4 HCC samples
    - CRLM ST
      - 4 CRLM samples

    - SpaceRanger software
      - Post-quality control

    - SCTtransform algorithm
      - data normalization
      - selection of highly variable genes
      - data transformation

    - average number of spots
      - 4054
    - average UMI
      - 15317.4
    - gene counts
      - 3890.1
    - mitochondrial content
      - 4.5%

    - Seurat
      - Analysis
      - visualization

    - CARD algorithm
      - conditional autoregression based deconvolution
      - Deconvolution analysis

    - single-cell annotations
      - cell type predictions for each spot

    - CARD software
      - Visualization of cell types in ST

    - Signature scores
      - Seurat
        - GSVA package
        - AddModelScore

## 50.1 sc Expression Profile

  - Markers

    - epithelial cell
      - EPCAM
      - KRT18
      - KRT19
      - CDH1

    - fibroblast
      - DCN
      - THY1
      - COL1A1
      - COL1A2

    - endothelial cell
      - PECAM1
      - CLDN5
      - FLT1
      - RAMP2

    - T cell 
      - CD3D
      - CD3E
      - CD3G
      - TRAC

    - NK cell 
      - NKG7
      - GNLY
      - NCAM1
      - KLRD1

    - B cell
      - CD79A
      - IGHM
      - IGHG3
      - IGHA2

    - mast cell 
      - KIT
      - MS4A2
      - GATA2

  - Pearson correlation matrix
    - correlation between two continuous variables
      - clustering results
      - cell type classification
    - marker gene expression
    - heatmap

  - Marker gene expression
    - heatmaps
    - tSNE plots
    - confirming cell-type identification

  - Comparisons
    - cell type composition
    - patient origin
    - cell count
    - transcript detection
    - CNV status

## 50.2 ST

  - HCC enriched
    - LIHCT
    - tumor cells
  - CRLM enriched
    - LIHC
    - fibroblasts
    - endothelial cells
    - myeloid cells

  - CNV detection
    - InferCNV software
    - CNV score distribution

  - deconvolution analysis
    - CARD algorithm
    - based on single-cell data
    - cell type distribution

    
  - H&E staining images

  - tumor vs. non-tumor classification

    - dimensionality reduction
    - clustering analysis



    - tumor cells
    8 subgroups
    LIHCT1
    LIHCT3
    LIHCT4

    tSNE plot
    subgroup composition
    sample origin
    CNV status
    patient composition

    Marker genes for each subgroup were calculated
    expression heatmaps were created



    - non-tumor cells
    5 subgroups
    endothelial cells
    tumor cells
    myeloid cells

    LIHCT2 enrichment

    tSNE plot
    marker gene expression heatmaps

## 50.3 Subclassification Analysis of Tumor Cells

Tumor cells were isolated and subjected to dimensionality reduction and clustering analysis, resulting in 16 clusters. These clusters were then merged based on their marker gene functions, yielding 7 tumor cell subgroups (Figure 3A). A bar plot displays the cell composition of each sample, showing that HCC samples have a higher content of APOA2+NEO cells, while CRLM samples have a higher content of TFF3+NEO cells. Differential gene expression analysis was performed for each subgroup, and a heatmap of differentially expressed genes was generated (Figure 3B). GO Biological Process (GOBP) enrichment analysis was conducted for the marker genes of each subgroup, revealing their corresponding functions. We compared the CNV differences among different cell types from different sample origins, finding significant differences in CNV across most cell types (Figure 3C). Transcription factor analysis was also performed for each cell type, with the results visualized in a heatmap showing different transcription factors enriched in various cell types (Figure 3D).

Subsequent cell trajectory analysis revealed multiple paths leading to different endpoints, indicating the potential existence of different subtypes. The starting branches contained a large number of APOA2+NEO cells and tumor cells from HCC (Figure 3E). From the pseudotime analysis, APOA2+NEO cells were primarily at the starting points, while most other cells were at the endpoints. Genes related to pseudotime were identified, and their expression heatmap was generated (Figure 3F). Scatter plots of pseudotime for six selected tumor marker genes (APOA2, HIST1H2AG, HMGB2, IGKV1-5, LCN2, TFF3) were created (Figure 3G).

## 50.3 Cell Annotation Analysis

  - tumor heterogeneity clustering
    - epithelial cells
    - immune cells
    - fibroblasts


## 50.4 Subgroup Analysis of Cell Populations

  - further distinction for subgroups
    - Immune cells
    - epithelial cells
    - fibroblasts

  - standard Seurat workflow
  - specific markers for each subgroup
  - UMAP plots

## 50.5 CNV Analysis of Single Cells

  - CNVs in tumor cell subgroups
    - InferCNV
    - references
      - fibroblasts
      - endothelial cells
    - identify malignant cells within subgroups

## 50.6 Pseudotime Analysis of Single Cells

  - Monocle2 software
    - pseudotime analysis
      - epithelial
      - macrophage
      - CAF cell

  - DDRTree
    - dimensionality reduction algorithm
    - parameters set to default
    - cell differentiation

## 50.7 Transcription Factor Analysis

  - SCENIC software
    - transcription factor analysis across cell subgroups
    - default parameters
      - RcisTarget
      - GRNBoost motif databases

  - RcisTarget package
    - overexpressed transcription factor binding motifs

  - AUCell package
    - scored the activity of each regulatory module for each cell type

## 50.8 Cell-Cell Communication Analysis

  - cell-cell communication
    - CellChat package
    - normalized gene expression matrix was imported

  - Data preprocessing
    - identifyOverExpressedGenes
    - identifyOverExpressedInteraction
    - ProjectData
    - default parameters

  - ligand-receptor interactions
    - computeCommunProb
    - filterCommunication
    - computeCommunProbPathway

  - cell communication network
    - aggregateNet function

## 50.9 Statistical Analysis

  - R 4.1.3
    - data processing
    - statistical analysis
    - plotting

  - Pearson correlation coefficient
    - correlation between two continuous variables

  - chi-square test
    - categorical variables

  - continuous variables
    - Wilcoxon rank-sum test
    - T-test

  - survminer package
    - determine optimal cutoff values

  - survival package
    - Cox regression
    - Kaplan-Meier analyses

Amino

In [56]:
# region
import os
import sqlite3
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random

# 连接到数据库并读取数据
def load_drug_data(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    #cursor.execute("SELECT Q, pathway, class FROM Pic WHERE class = 'Amino_1' or class = 'Amino_2' or class = 'Amino_3'")
    cursor.execute("SELECT Q, pathway, class FROM Pic WHERE class = 'Amino_3'")
    data = cursor.fetchall()
    conn.close()
    return data

def create_table(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists
    cursor.execute('''
    DROP TABLE IF EXISTS Pic
    ''')
    
    # Create the table
    cursor.execute('''
    CREATE TABLE Pic (
        Q TEXT PRIMARY KEY,
        pathway TEXT,
        class TEXT
    )
    ''')
    
    conn.commit()
    conn.close()

# 假设数据库路径为 d:\\desktop\\Mein Kampf\\Guerrilla.db
db_path = r'd:\desktop\Mein Kampf\Guerrilla.db'
create_table(db_path)
data = load_drug_data(db_path)

# 初始化药物结构字典，包含 class 信息
drugs_structures = {}
for row in data:
    q, pathway, class_name = row
    if class_name not in drugs_structures:
        drugs_structures[class_name] = {}
    drugs_structures[class_name][q] = pathway

# 初始化连续答对的计数器
correct_streak = 0
target_streak = 20

def insert_data(db_path, q, pathway, class_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR REPLACE INTO Pic (Q, pathway, class)
    VALUES (?, ?, ?)
    ''', (q, pathway, class_name))
    conn.commit()
    conn.close()

def read_folder_and_insert_data(base_folder, db_path):
    for class_name in os.listdir(base_folder):
        class_path = os.path.join(base_folder, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    q, _ = os.path.splitext(file_name)  # 去除文件后缀
                    pathway = file_path
                    insert_data(db_path, q, pathway, class_name)

def generate_question_gui(window):
    global correct_streak  # 声明使用全局变量

    # 清除窗口中现有的所有组件
    for widget in window.winfo_children():
        widget.destroy()

    # 随机选取一个类别，并在该类别中选择正确答案
    class_name = random.choice(list(drugs_structures.keys()))
    correct_drug, correct_link = random.choice(list(drugs_structures[class_name].items()))

    # 从相同类别中移除正确答案，以便从剩余药物中选取错误选项
    temp_structures = drugs_structures[class_name].copy()
    del temp_structures[correct_drug]

    # 随机选择三个错误选项
    wrong_links = random.sample(list(temp_structures.values()), 3)
    all_options = wrong_links + [correct_link]
    random.shuffle(all_options)  # 打乱所有选项的顺序

    # 创建一个Label来显示药物名称
    tk.Label(window, text=f"Q：{correct_drug}", font=("Arial", 16)).pack()

    # 创建一个Frame来放置选项按钮
    frame = tk.Frame(window)
    frame.pack(pady=20)

    # 定义一个函数来处理用户的选择
    def handle_choice(index):
        global correct_streak  # 再次声明使用全局变量
        if all_options[index] == correct_link:
            correct_streak += 1
            #print("正确！")
        else:
            correct_index = all_options.index(correct_link) + 1
            messagebox.showerror("结果", f"回答错误。\n正确的结构式是选项 {correct_index}。\n你之前连续答对了 {correct_streak} 题。")
            correct_streak = 0

        if correct_streak >= target_streak:
            window.destroy()
        else:
            generate_question_gui(window)  # 生成新的题目

    # 为每个选项创建一个按钮，并附上图片
    for i, option in enumerate(all_options):
        try:
            # 加载图片
            img = Image.open(option)
            # 调整图片大小并保持长宽比，限制在600x75范围内
            img.thumbnail((600, 300), Image.Resampling.LANCZOS)
            # 创建一个新的白色背景图片
            background = Image.new('RGB', (600, 300), (255, 255, 255))
            # 计算图像放置的位置，使其居中
            img_w, img_h = img.size
            bg_w, bg_h = background.size
            offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
            # 将缩放后的图像粘贴到白色背景上
            background.paste(img, offset)
            img = ImageTk.PhotoImage(background)
            button = tk.Button(frame, image=img, command=lambda i=i: handle_choice(i))
            button.img = img  # 保留对图片的引用
            button.grid(row=i//2, column=i%2, padx=10, pady=10)  # 2*2布局，并添加间距
        except Exception as e:
            print(f"无法加载图片 {option}: {e}")

    # 添加一个退出按钮
    tk.Button(window, text="退出", command=window.quit, font=("Arial", 14)).pack(pady=10)

if __name__ == '__main__':
    base_folder = r'D:\desktop\Mein Kampf\Pic'  # 替换为实际的基准文件夹路径
    db_path = r'd:\desktop\Mein Kampf\Guerrilla.db'  # 替换为实际的数据库路径
    create_table(db_path)
    read_folder_and_insert_data(base_folder, db_path)
    
    # 更新药物结构字典
    data = load_drug_data(db_path)
    drugs_structures = {}
    for row in data:
        q, pathway, class_name = row
        if class_name not in drugs_structures:
            drugs_structures[class_name] = {}
        drugs_structures[class_name][q] = pathway

    # 创建图形界面窗口
    window = tk.Tk()
    window.title("药物结构式识别 Quiz")

    # 第一次生成题目
    generate_question_gui(window)

    # 运行图形界面的主循环
    window.mainloop()
# endregion